# Practicum 5: Classification



## Task 1: Learning a Decision Tree 

  - Obtain the impurity measurements of a node. 
  - Implement simple splitting procedures.
  - Calculate the information gain for possible splittings, and decide for the best. 

### Preliminaries

In this task we work with the dataset used in the exercise from the last lecture.
Given observations on some weather properties, we learnt a decision tree to predict if we can play outside.

We will use the **csv** module for reading in data from a file.

In [3]:
import csv

The data set is, as usual, stored in a comma-separated text file.

We read it and store it as a list of records, where each record is represented using a dict.

In [4]:
def load_play_data(filename):
    records = []
    with open(filename, 'rt') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        for row in csvreader:
            if len(row) == 5:  # if we have 4 fields in that line
                records.append({
                    "Outlook": row[0],
                    "Temp": float(row[1]),
                    "Humidity": float(row[2]),
                    "Windy": row[3],
                    "Target": row[4]
                })
    return records

play_data = load_play_data("data/dt_play.data.csv")

We can get all the possible values for the attribute "Outlook".

In [5]:
set([x["Outlook"] for x in play_data])

{'overcast', 'rain', 'sunny'}

We can also select some records by their ID, to be the position by the order in the file.

For example, we can select the records to be associated with the child of the root reached by the condition "sunny" if it is splitting by "Outlook" (i.e., records for the call 2.1 in the exercise).

In [6]:
[play_data[i] for i in range(len(play_data)) if i in {0, 1, 7, 8, 10}]  # {1, 2, 8, 9, 11} shifted one to the left

[{'Humidity': 85.0,
  'Outlook': 'sunny',
  'Target': 'No',
  'Temp': 85.0,
  'Windy': 'false'},
 {'Humidity': 90.0,
  'Outlook': 'sunny',
  'Target': 'No',
  'Temp': 80.0,
  'Windy': 'true'},
 {'Humidity': 95.0,
  'Outlook': 'sunny',
  'Target': 'No',
  'Temp': 72.0,
  'Windy': 'false'},
 {'Humidity': 70.0,
  'Outlook': 'sunny',
  'Target': 'Yes',
  'Temp': 69.0,
  'Windy': 'false'},
 {'Humidity': 70.0,
  'Outlook': 'sunny',
  'Target': 'Yes',
  'Temp': 75.0,
  'Windy': 'true'}]

This records list is, of course, the same as this one, just selecting the records by that attribute value from the entire dataset.

In [7]:
#[x for x in play_data if x["Outlook"] == "sunny"]

### Impurity Measures: Entropy

Define the entropy of (the classes distribution of a list of records associated with) a node.

The **entropy** for a node $t$ and $c$ classes is $\mathrm{Entropy}(t) = -\sum_{i=0}^{c-1}P(i|t)log_2 P(i|t)$ .

In [1]:
import math
from collections import Counter

def entropy(records):
    """Entropy of a list of records associated with a node."""
    count = Counter([x["Target"] for x in records])
    return sum([(-freq / len(records)) * math.log(freq / len(records), 2) for freq in count.values()])

Obtain the entropy of the entire dataset (done in the exercise for the records associated to the root).

In [8]:
entropy(play_data)

0.9402859586706309

### Splitting by a categorical attribute

Define the following function to split a list of records by a given **categorical** attribute, using a partition of possible values as outcomes to the children.

In [12]:
def split_categorical(records, attr_name, values_sets):
    """Splits a list of records by a given categorical attribute, using a partition of possible values.
    
    :param records: a list of records to split.
    :param attr_name: the name of the categorical attribute.
    :param values_sets: a partition list of sets A_i. Given the m possible values Vals(D) = {dj | j=1, ..., m} of the \
    attribute D, each A_i is a subset of possible values, such that all the A_i sets make a partition (mutually \
    disjoint, and the union of all is Vals(D)).
    For example, when D="Outlook", Vals(D) = {"sunny", "overcast", "rain"}, then a possible partition is \
    values_sets = [{"sunny"}, {"overcast", "rain"}].
    Another possible partition is values_sets = [{"sunny"}, {"overcast"}, {"rain"}].
    :return: a list of lists, each of these ones contains all the records associated with one of the children.
    """
    print("Splitting by {}".format(attr_name))
    children = []
    for a_i in values_sets:  # for each subset of possible values
        child = [x for x in records if x[attr_name] in a_i]  # 
        children.append(child)
        
        # e.g. if values_sets = [{"sunny"}, {"overcast", "rain"}], and atr_name = "Outlook"
        # then, in the 2nd iteration, a_i = {"overcast", "rain"},
        # so child = list of records for which the value in "Outlook" attr is in {"overcast", "rain"}
        
        # We also print the entropy for each child
        print("\tChild condition: {}\tSize = {}\tEntropy = {}".format(a_i, len(child), entropy(child)))
    return children

Obtain the records associated with each the children nodes after splitting the entire dataset by "Outlook" with its three possible values as partitions (just like in the exercise).

In [13]:
children = split_categorical(play_data, "Outlook", [{"sunny"}, {"overcast"}, {"rain"}])

Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Entropy = 0.9709505944546686
	Child condition: {'overcast'}	Size = 4	Entropy = 0.0
	Child condition: {'rain'}	Size = 5	Entropy = 0.9709505944546686


### Impurity Measures: Gini and Classification Error

Define the Gini impurity of (the classes distribution of a list of records associated with) a node.

The **Gini impurity** for a node $t$ and $c$ classes is $\mathrm{Gini}(t) = 1-\sum_{i=0}^{c-1}P(i|t)^2$ .

In [14]:
def gini(records):
    """Gini impurity of a list of records associated with a node."""
    count = Counter([x["Target"] for x in records])
    return 1 - sum([math.pow(freq / len(records), 2) for freq in count.values()])

We can verify the consistency with respect to entropy as a measurement of the impurity of the entire dataset. For this, get the Gini impurity for each child, and compare with the respective entropy measurements.
  

In [16]:
# Same definition as for entropy but printing Gini inside
def split_categorical_showing_gini(records, attr_name, values_sets):
    """Splits a list of records by a given categorical attribute, using a partition of possible values.
    Shows Gini impurity during calculations.
    
    :param records: a list of records to split.
    :param attr_name: the name of the categorical attribute.
    :param values_sets: a partition list of sets A_i. Given the m possible values Vals(D) = {dj | j=1, ..., m} of the \
    attribute D, each A_i is a subset of possible values, such that all the A_i sets make a partition (mutually \
    disjoint, and the union of all gives Vals(D)).
    For example, when D="Outlook", Vals(D) = {"sunny", "overcast", "rain"}, then a possible partition is \
    values_sets = [{"sunny"}, {"overcast", "rain"}].
    Another possible partition is values_sets = [{"sunny"}, {"overcast"}, {"rain"}].
    :return: a list of lists, each list contains all the records associated with one child.
    """
    print("Splitting by {}".format(attr_name))
    children = []
    for a_i in values_sets:  # for each subset of possible values
        child = [x for x in records if x[attr_name] in a_i]  # 
        children.append(child)
        
        # e.g. if values_sets = [{"sunny"}, {"overcast", "rain"}], and atr_name = "Outlook"
        # then, in the 2nd iteration, a_i = {"overcast", "rain"},
        # so child = list of records for which the value in "Outlook" attr is in {"overcast", "rain"}
        
        # We also print the Gini impurity for each child
        print("\tChild condition: {}\tSize = {}\tGini impurity = {}".format(a_i, len(child), gini(child)))
    return children

children = split_categorical_showing_gini(play_data, "Outlook", [{"sunny"}, {"overcast"}, {"rain"}])

Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Gini impurity = 0.48
	Child condition: {'overcast'}	Size = 4	Gini impurity = 0.0
	Child condition: {'rain'}	Size = 5	Gini impurity = 0.48


In [32]:
# Or simply like this:
children = split_categorical(play_data, "Outlook", [{"sunny"}, {"overcast"}, {"rain"}])
for i, child in enumerate(children):
    print("{}-th child. Gini impurity = {}".format(i, gini(child)))
    
gini(play_data)

Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Entropy = 0.9709505944546686
	Child condition: {'overcast'}	Size = 4	Entropy = 0.0
	Child condition: {'rain'}	Size = 5	Entropy = 0.9709505944546686
0-th child. Gini impurity = 0.48
1-th child. Gini impurity = 0.0
2-th child. Gini impurity = 0.48


0.4591836734693877

Define the classification error impurity of (the classes distribution of a list of records associated with) a node.

The **classification error** for a node $t$ and $c$ classes is $\mathrm{Classification~error}(t) = 1-\max P(i|t)$ .

In [19]:
def classif_error(records):
    """Classification error of a list of records associated with a node."""
    count = Counter([x["Target"] for x in records])
    return 1 - max([freq / len(records) for freq in count.values()])

Similarly as before, we can verify the consistency of the classification error with respect to the other impurities, in particular on the entire dataset. For this, get the classification error for each child, and compare with the respective measurements for previous impurities.

In [20]:
children = split_categorical(play_data, "Outlook", [{"sunny"}, {"overcast"}, {"rain"}])
for i, child in enumerate(children):
    print("{}-th child. Size = {}\tClassification error = {}".format(i, len(child), classif_error(child)))

Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Entropy = 0.9709505944546686
	Child condition: {'overcast'}	Size = 4	Entropy = 0.0
	Child condition: {'rain'}	Size = 5	Entropy = 0.9709505944546686
0-th child. Size = 5	Classification error = 0.4
1-th child. Size = 4	Classification error = 0.0
2-th child. Size = 5	Classification error = 0.4


### Splitting by a numeric attribute

Let's implement a simple way to split a **numeric** attribute in a binary split. As in the exercise, a fixed splitting point will be provided to lead to two children nodes:
  - one in which all the records have for that attribute a value less than or equal to the splitting point,
  - and the other one with the attribute value greater than the spitting point.

In [27]:
def split_numeric_binary(records, attr_name, splitting_point):
    """Splits in a binary way -i.e., using a single splitting point- a list of records by a given numeric attribute.

    :param records: a list of records to split.
    :param attr_name: the name of the numeric attribute.
    :param splitting_point: a single number to be splitting point, such that one child will have the records \
    for which the attribute value is less or equal than splitting_point, and the other the ones with values greater \
    than splitting_point.
    E.g., 75 was the splitting point for the attrribute "Temp" and also for "Humidity".
    :return: a list of lists, each list contains all the records associated with one child.
    """
    print("Splitting by {}".format(attr_name))
    children = [[x for x in records if x[attr_name] <= splitting_point],
               [x for x in records if x[attr_name] > splitting_point]]
    
    # We also print the entropy for each child
    print("\t'Less-or-equal-than' child. Size = {}\tEntropy = {}".format(len(children[0]), entropy(children[0])))
    print("\t'Greater-than' child. Size = {}\tEntropy = {}".format(len(children[1]), entropy(children[1])))

    return children

Obtain the records associated with each the children nodes after splitting the entire dataset by "Humidity" with 75 as the splitting point (just like in the exercise attempt).

In [28]:
children = split_numeric_binary(play_data, "Humidity", 75)

Splitting by Humidity
	'Less-or-equal-than' child. Size = 5	Entropy = 0.7219280948873623
	'Greater-than' child. Size = 9	Entropy = 0.9910760598382222


### Information gain

After seen that all the impurity measures behave similarly, we come back to use entropy so that we can define the **information gain** (the formula for it is in the exercise description).

In [29]:
def infogain(parent_records, children_records):
    """Information gain between a node and its children nodes.
    
    :param parent_records: list of records associated with the parent node.
    :param children_records: list of lists, each list contains all the records associated with one child.
    """
    entropy_p = entropy(parent_records)
    return entropy_p - sum([(len(child_records) / len(parent_records)) * entropy(child_records)
                                for child_records in children_records])

Get the information gain between the root and the children nodes after splitting the entire dataset by "Outlook" with its three possible values as partitions (just like in the exercise).

In [30]:
children = split_categorical(play_data, "Outlook", [{"sunny"}, {"overcast"}, {"rain"}])
infogain(play_data, children)

Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Entropy = 0.9709505944546686
	Child condition: {'overcast'}	Size = 4	Entropy = 0.0
	Child condition: {'rain'}	Size = 5	Entropy = 0.9709505944546686


0.2467498197744391

Compute the first splitting of the records at the root node as in the exercise. This is, write a small code that assesses the possible splittings by each of the attributes (with the splitting criteria as in the exercise, e.g., all the three values for "Outlook" without grouping, and 75 as splitting point for the numerical attributes) and shows the decreasingly sorted information gains.

In [31]:
from operator import itemgetter

CATEGORICAL_ATTRS = {"Outlook", "Windy"}
NUMERIC_ATTRS = {"Temp", "Humidity"}

attr_criteria = {"Outlook": [{"sunny"}, {"overcast"}, {"rain"}],
                "Temp": 75,
                "Humidity": 75,
                "Windy": [{"true"}, {"false"}]
                }

attr_igs = {}
for attr_name, criterion in attr_criteria.items():
    children = (split_categorical(play_data, attr_name, criterion) if attr_name in CATEGORICAL_ATTRS
                else split_numeric_binary(play_data, attr_name, criterion))
    ig = infogain(play_data, children)
    attr_igs[attr_name] = ig

for attr_name, ig in sorted(attr_igs.items(), key=itemgetter(1), reverse=True):
    print("Info gain if splitting by {}: {}".format(attr_name, ig))

Splitting by Humidity
	'Less-or-equal-than' child. Size = 5	Entropy = 0.7219280948873623
	'Greater-than' child. Size = 9	Entropy = 0.9910760598382222
Splitting by Outlook
	Child condition: {'sunny'}	Size = 5	Entropy = 0.9709505944546686
	Child condition: {'overcast'}	Size = 4	Entropy = 0.0
	Child condition: {'rain'}	Size = 5	Entropy = 0.9709505944546686
Splitting by Temp
	'Less-or-equal-than' child. Size = 10	Entropy = 0.8812908992306927
	'Greater-than' child. Size = 4	Entropy = 1.0
Splitting by Windy
	Child condition: {'true'}	Size = 6	Entropy = 1.0
	Child condition: {'false'}	Size = 8	Entropy = 0.8112781244591328
Info gain if splitting by Outlook: 0.2467498197744391
Info gain if splitting by Windy: 0.04812703040826927
Info gain if splitting by Humidity: 0.04533417202914425
Info gain if splitting by Temp: 0.0250781735058504
